In [7]:
import numpy as np
import torch

from torch_obb.estimation import obb_estimate_pca, obb_estimate_dito

rng = np.random.RandomState(42)

def sample_spherical(npoints, ndim=3):
    vec = np.random.randn(npoints, ndim).astype(np.float32)
    vec /= np.linalg.norm(vec, axis=1)[:, None]
    return vec

points1 = rng.random((500, 3)).astype(np.float32) * np.array([1, 1, 1], dtype=np.float32)
points2 = rng.random((14, 3)).astype(np.float32) * np.array([1, 5, 2], dtype=np.float32)
points3 = sample_spherical(100, 3) * np.array([1, 2, 1], dtype=np.float32)

points = torch.nested.as_nested_tensor((torch.from_numpy(points1), torch.from_numpy(points2), torch.from_numpy(points3)), layout=torch.jagged)
vertices_pca = obb_estimate_pca(points).squeeze()

In [8]:
vertices_dito = obb_estimate_dito(points).squeeze()

In [9]:
import pygfx as gfx
import trimesh
from rendercanvas.glfw import GlfwRenderCanvas

_TRIS = [[1, 3, 0],
         [4, 1, 0],
         [0, 3, 2],
         [2, 4, 0],
         [1, 7, 3],
         [5, 1, 4],
         [5, 7, 1],
         [3, 7, 2],
         [6, 4, 2],
         [2, 7, 6],
         [6, 5, 4],
         [7, 5, 6]]
i = 2
canvas = GlfwRenderCanvas(size=(1024, 768), max_fps=60, update_mode="continuous")   
scene = gfx.Scene()
background = gfx.Background(None, gfx.BackgroundMaterial('white'))
scene.add(background)

vertices_pca_np = vertices_pca[i].cpu().numpy()
vertices_dito_np = vertices_dito[i].cpu().numpy()
points_np = points[i].cpu().numpy()


points_geometry = gfx.Geometry(positions=points_np.astype(np.float32))
pc = gfx.Points(points_geometry, material=gfx.PointsMaterial(size=10, color=(0, 0, 0, 1.0)))
scene.add(pc)
#box_local = gfx.BoxHelper(thickness=2, color="green")
#box_local.set_transform_by_object(pc, space="local")
#pc.add(box_local)  # note that the parent is `mesh` here, not `scene`

obb_geometry_pca = gfx.Geometry(positions=vertices_pca_np.squeeze().astype(np.float32), indices=np.array(_TRIS, dtype=np.int32))
obb_pca = gfx.Mesh(obb_geometry_pca, gfx.MeshBasicMaterial(wireframe=True, color='cyan', wireframe_thickness=2))
scene.add(obb_pca)

obb_geometry_dito = gfx.Geometry(positions=vertices_dito_np.squeeze().astype(np.float32), indices=np.array(_TRIS, dtype=np.int32))
obb_dito = gfx.Mesh(obb_geometry_dito, gfx.MeshBasicMaterial(wireframe=True, color='red', wireframe_thickness=2))
scene.add(obb_dito)


disp = gfx.Display(canvas=canvas)
disp.show(scene)

canvas.is_closed() is deprecated, use canvas.get_closed() instead.
Your scene does not contain any lights. Some objects may not be visible.
Unrecognized present mode 1000361000
Unrecognized present mode 1000361000


Unrecognized present mode 1000361000
